In [ ]:
!pip install gradio sentence-transformers deep-translator langdetect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
# 2) استيراد المكتبات
import gradio as gr
from sentence_transformers import SentenceTransformer, util
from langdetect import detect
from deep_translator import GoogleTranslator


In [ ]:
# تحميل نموذج متعدد اللغات
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# 3) قاعدة بيانات الهوايات (بالإنجليزي فقط كـ base)
# ================================
hobbies_data = {
    "football": ["Play a match with your friends", "Watch football training videos", "Try to learn new dribbling skills"],
    "drawing": ["Draw a landscape", "Try coloring a sketch", "Design a greeting card for your family"],
    "reading": ["Read a short story", "Start a new book", "Tell someone the story you read"],
    "music": ["Learn a simple melody", "Listen to calm music", "Try composing a short song"]
}

# تحويل أسماء الهوايات إلى Embeddings
hobby_names = list(hobbies_data.keys())
hobby_embeddings = model.encode(hobby_names, convert_to_tensor=True)


In [ ]:
# 4) دالة البوت
def hobby_bot(user_input):
    # نحدد لغة المستخدم
    try:
        lang = detect(user_input)
    except:
        lang = "en"  # لو معرفهاش نخليها إنجليزي

    # Embedding للجملة
    input_embedding = model.encode(user_input, convert_to_tensor=True)

    # حساب التشابه
    similarities = util.pytorch_cos_sim(input_embedding, hobby_embeddings)[0]
    best_match_idx = int(similarities.argmax())
    best_score = float(similarities[best_match_idx])

    if best_score < 0.5:
        msg_en = "❓ I didn’t understand your hobby clearly, could you tell me more?"
        # نترجم الرسالة للغة المستخدم
        msg_translated = GoogleTranslator(source="en", target=lang).translate(msg_en)
        return msg_translated

    hobby = hobby_names[best_match_idx]
    suggestions = hobbies_data[hobby]

    # نجهز الرد بالإنجليزي الأول
    msg_en = f"It looks like you love {hobby} ✨\nHere are some ideas:\n- " + "\n- ".join(suggestions)

    # نترجم الرسالة إلى لغة المستخدم
    msg_translated = GoogleTranslator(source="en", target=lang).translate(msg_en)
    return msg_translated


In [ ]:
# 5) واجهة Gradio
iface = gr.Interface(
    fn=hobby_bot,
    inputs="text",
    outputs="text",
    title="🌍 Multilingual AI Hobbies Bot",
    description="Tell me your hobby in ANY language, and I'll suggest fun activities in the same language!"
)

# 6) تشغيل الواجهة
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3408fcda20fdadc8ad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
